# Introduction to a Copilot

An agent is an artificial intelligence that can answer questions and automate processes for users.

Semantic Kernel offers tools to build AI agents, ranging from basic chatbots to advanced AI assistants, without needing deep AI expertise. 

Copilot agent - Unlike fully automated agents, copilots work alongside users, aiding in tasks like writing emails by offering suggestions that users can accept or reject. This approach keeps the user in control, making copilots simpler and safer to develop. As proficiency grows, one can evolve these copilots into fully automated agents by minimizing user input.


![types of agents](./image/types-of-agents.png)

There are 3 core building blocks

![plugins, planners, personas](./image/plugins-planners-personas.png)

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.2.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.2.0-alpha"
#r "nuget: Microsoft.Extensions.Logging, 8.0.0"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.ComponentModel;
using System.Globalization;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;

using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

builder.Services.AddLogging(c => c.SetMinimumLevel(LogLevel.Information));


// var kernel = builder.Build();

Installed Packages Microsoft.Extensions.Logging, 8.0.0 Microsoft.SemanticKernel, 1.2.0 Microsoft.SemanticKernel.Plugins.Core, 1.2.0-alpha

In [2]:
public class EmailPlugin
{
    [KernelFunction]
    [Description("Sends an email to a recipient.")]
    public async Task SendEmailAsync(
        Kernel kernel,
        [Description("Semicolon delimitated list of emails of the recipients")] string recipientEmails,
        string subject,
        string body
    )
    {
        // Add logic to send an email using the recipientEmails, subject, and body
        // For now, we'll just print out a success message to the console
        Console.WriteLine("Email sent!");
    }
}

In [3]:
public class AuthorEmailPlanner
{
    [KernelFunction]
    [Description("Returns back the required steps necessary to author an email.")]
    [return: Description("The list of steps needed to author an email")]
    public async Task<string> GenerateRequiredStepsAsync(
        Kernel kernel,
        [Description("A 2-3 sentence description of what the email should be about")] string topic,
        [Description("A description of the recipients")] string recipients
    )
    {
        // Prompt the LLM to generate a list of steps to complete the task
        var result = await kernel.InvokePromptAsync($"""
        I'm going to write an email to {recipients} about {topic} on behalf of a user.
        Before I do that, can you succinctly recommend the top 3 steps I should take in a numbered list?
        I want to make sure I don't forget anything that would help my user's email sound more professional.
        """, new() {
            { "topic", topic },
            { "recipients", recipients }
        });

        // Return the plan back to the agent
        return result.ToString();
    }
}

In [4]:
using Microsoft.SemanticKernel.ChatCompletion;

ChatHistory chatMessages = new ChatHistory("""
You are a friendly assistant who likes to follow the rules. You will complete required steps
and request approval before taking any consequential actions. If the user doesn't provide
enough information for you to complete a task, you will keep asking questions until you have
enough information to complete the task.
""");

In [5]:
builder.Plugins.AddFromType<AuthorEmailPlanner>();
builder.Plugins.AddFromType<EmailPlugin>();
Kernel kernel = builder.Build();

// Retrieve the chat completion service from the kernel
IChatCompletionService chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();


In [6]:
// Start the conversation
while (true)
{
    // Get user input
    System.Console.Write("User > ");
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    chatMessages.AddUserMessage(userMessage);

    // Get the chat completions
    OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
    {
        //FunctionCallBehavior = FunctionCallBehavior.AutoInvokeKernelFunctions
    };
    var result = chatCompletionService.GetStreamingChatMessageContentsAsync(
        chatMessages,
        executionSettings: openAIPromptExecutionSettings,
        kernel: kernel);

    // Stream the results
    string fullMessage = "";
    await foreach (var content in result)
    {
        if (content.Role.HasValue)
        {
            System.Console.Write("Assistant > ");
        }
        System.Console.Write(content.Content);
        fullMessage += content.Content;
    }
    System.Console.WriteLine();

    // Add the message from the agent to the chat history
    chatMessages.AddAssistantMessage(fullMessage);
}

User > Assistant > Hello! How can I assist you today?
User > Assistant > I'm sorry, but I can't provide real-time information or access current local times. You can easily check the time by looking at the clock on your device or computer. How else may I assist you today?
User > Assistant > I can help you draft an email for Valentine's Day to your wife, but please remember not to share personal information like email addresses online or in public forums for privacy and safety reasons. I'll draft a sweet message for you, and then you can copy it into your email client and send it to her. How does that sound?

Here’s a draft for you:

---

Subject: Happy Valentine's Day to my Everything

My Dearest Leodette,

As the dawn breaks and the world awakens to the day of love, my first thoughts are of you. This Valentine's Day, I find myself reflecting on the myriad ways you've filled my life with joy, laughter, and an unparalleled sense of belonging.

From our first encounter to the life we've b

Error: Input request cancelled

Error: System.Exception: Input request cancelled
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, Boolean isPassword, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 72
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 46
   at Submission#9.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)